In [9]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
model_name = 'keras_cifar10_trained_model-2.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [10]:
# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same",
                 input_shape=x_train.shape[1:], activation="relu"))
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation="softmax"))

# initiate RMSprop optimizer
opt = RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test))

# Save model and weights
model.save(model_name)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Using real-time data augmentation.
Epoch 1/100
1563/1563 [==============================] - 75s 48ms/step - loss: 1.8909 - accuracy: 0.3063 - val_loss: 1.6238 - val_accuracy: 0.4034
Epoch 2/100
1563/1563 [==============================] - 73s 47ms/step - loss: 1.6149 - accuracy: 0.4115 - val_loss: 1.4076 - val_accuracy: 0.4924
Epoch 3/100
1563/1563 [==============================] - 74s 47ms/step - loss: 1.4830 - accuracy: 0.4642 - val_loss: 1.3653 - val_accuracy: 0.5105
Epoch 4/100
1563/1563 [==============================] - 74s 48ms/step - loss: 1.4003 - accuracy: 0.4989 - val_loss: 1.3908 - val_accuracy: 0.5166
Epoch 5/100
1563/1563 [==============================] - 74s 48ms/step - loss: 1.3308 - accuracy: 0.5240 - val_loss: 1.2348 - val_accuracy: 0.5577
Epoch 6/100
1563/1563 [==============================] - 74s 47ms/step - loss: 1.2761 - accuracy: 0.5432 - val_loss: 1.1841 - val_accuracy: 0.5827
Epoch 7/100
1563/1563 [==============================] - 74s 47ms/step - loss: 1.23

1563/1563 [==============================] - 71s 46ms/step - loss: 0.7924 - accuracy: 0.7339 - val_loss: 0.7491 - val_accuracy: 0.7580
Epoch 57/100
1563/1563 [==============================] - 71s 45ms/step - loss: 0.7927 - accuracy: 0.7327 - val_loss: 0.7775 - val_accuracy: 0.7610
Epoch 58/100
1563/1563 [==============================] - 71s 45ms/step - loss: 0.7866 - accuracy: 0.7341 - val_loss: 0.6718 - val_accuracy: 0.7728
Epoch 59/100
1563/1563 [==============================] - 72s 46ms/step - loss: 0.7927 - accuracy: 0.7315 - val_loss: 0.6842 - val_accuracy: 0.7696
Epoch 60/100
1563/1563 [==============================] - 71s 45ms/step - loss: 0.7910 - accuracy: 0.7339 - val_loss: 0.6827 - val_accuracy: 0.7720
Epoch 61/100
1563/1563 [==============================] - 72s 46ms/step - loss: 0.7891 - accuracy: 0.7340 - val_loss: 0.7573 - val_accuracy: 0.7616
Epoch 62/100
1563/1563 [==============================] - 71s 46ms/step - loss: 0.7918 - accuracy: 0.7331 - val_loss: 0.7463 

In [5]:
L = model.layers[0]

In [6]:
L.name

'conv2d'

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 13, 13, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0